<a href="https://colab.research.google.com/github/arya23-dev/BislabArya/blob/main/parallelcellular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

# Define the update rule for the cellular automaton
def update_cell(left, center, right):
    """Example rule: Majority rule (cell becomes 1 if at least 2 neighbors are 1)."""
    return int((left + center + right) >= 2)

# Function to process a subgrid chunk
def process_chunk(start_idx, end_idx, grid, next_grid, ncols):
    """Process a chunk of the grid and update its states."""
    for i in range(start_idx, end_idx):
        # Handle periodic boundary conditions (wrap around edges)
        left = grid[(i - 1) % ncols]
        center = grid[i]
        right = grid[(i + 1) % ncols]

        next_grid[i] = update_cell(left, center, right)

# Function to perform parallel computation on the grid
def parallel_cellular_automaton(ncols, nsteps, num_workers):
    """Run the parallel cellular automaton."""
    # Initialize grid with random 0s and 1s
    grid = np.random.randint(2, size=ncols)
    next_grid = np.zeros_like(grid)

    # Compute chunk size for each worker
    chunk_size = ncols // num_workers
    chunks = [(i * chunk_size, (i + 1) * chunk_size if i < num_workers - 1 else ncols) for i in range(num_workers)]

    # Create a process pool for parallel execution
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        for step in range(nsteps):
            # Submit tasks to the executor for parallel processing of chunks
            futures = []
            for start_idx, end_idx in chunks:
                futures.append(executor.submit(process_chunk, start_idx, end_idx, grid, next_grid, ncols))

            # Wait for all processes to finish
            for future in futures:
                future.result()

            # Swap grids (current and next)
            grid, next_grid = next_grid, grid

    return grid

# Main function to run the automaton
if __name__ == "__main__":
    ncols = 100  # Number of cells in the grid
    nsteps = 50  # Number of steps (iterations) for the cellular automaton
    num_workers = 4  # Number of parallel workers (processes)

    final_grid = parallel_cellular_automaton(ncols, nsteps, num_workers)
    print("Final Grid State:")
    print(final_grid)

Final Grid State:
[0 1 1 1 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0
 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 0
 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0]
